In [6]:
import json
import re
from collections import defaultdict
from dataclasses import dataclass, field
from pathlib import Path

from graphviz import Digraph
from slpp import slpp as lua

In [7]:
DIFFICULTY = "normal"
RECIPE_FILE = "data/recipe.lua"
ICON_DIR = "data/icons-cropped/"

# Thes are speeds of the machines we assume for crafting and smelting
CRAFTING_SPEED = 1.25  # Assembling machine 3
SMELTING_SPEED = 2  # Electric furnace

RECIPE_SPEEDS = {
    "advanced-crafting": CRAFTING_SPEED,
    "crafting": CRAFTING_SPEED,
    "crafting-with-fluid": CRAFTING_SPEED,
    "smelting": SMELTING_SPEED,
    "chemistry": 1.0,  # Chemical plant
    "rocket-building": 1.0,  # Rocket silo
    "centrifuging": 1.0,  # Centrifuge
    "oil-processing": 1.0,  # Oil refinery
}

In [8]:
text = Path(RECIPE_FILE).read_text()

# Remove wrapper
text = text.removeprefix("data:extend(\n").removesuffix(")\n")
text = text.strip()

# Remove comments
text = re.sub(r"--.*", "", text)

raw_recipes = lua.decode(text)

Path("data/recipe.json").write_text(json.dumps(raw_recipes, indent=2))

raw_recipe_keys = raw_recipe_keys = {key for recipe in raw_recipes for key in recipe.keys()}

print(len(raw_recipes))
print(raw_recipe_keys)

198
{'crafting_machine_tint', 'expensive', 'hidden', 'type', 'result', 'icon_size', 'name', 'icon_mipmaps', 'ingredients', 'normal', 'allow_decomposition', 'subgroup', 'main_product', 'icon', 'result_count', 'order', 'energy_required', 'requester_paste_multiplier', 'results', 'enabled', 'category'}


In [15]:
@dataclass
class Recipe:
    name: str
    category: str = "crafting"
    energy_required: float = 0.5
    ingredients: dict[str, int] = field(default_factory=dict)
    result_count: float = 1


recipes: dict[str, Recipe] = {}
for raw_recipe in raw_recipes:
    name = raw_recipe["name"]
    category = raw_recipe.get("category", "crafting")
    amount_dict = raw_recipe.get(DIFFICULTY, raw_recipe)

    # Parse out the ingredients
    ingredients: dict[str, int] = {}
    raw_ingredients = amount_dict.get("ingredients")
    for raw_ingredient in raw_ingredients:
        if isinstance(raw_ingredient, list):
            ingredient_name, ingredient_amount = raw_ingredient
        elif isinstance(raw_ingredient, dict):
            ingredient_name = raw_ingredient.get("name")
            ingredient_amount = raw_ingredient.get("amount")
        else:
            print(f"Issue with ingredients for `{name}`")
            continue
        ingredients[ingredient_name] = ingredient_amount

    # Parse out the results
    raw_result = amount_dict.get("result")
    if raw_result is None:
        print(f"Skipping {name}")
        continue
    if not isinstance(raw_result, str) or raw_result != name:
        print(raw_result, name, "is weird, will skip")
        continue

    # # This logic is for fluid processing stuff, don't care about it right now 
    # elif raw_result is None and isinstance(raw_results, list):
    #     for raw_result in raw_results:
    #         if isinstance(raw_result, list):
    #             result_name, result_amount = raw_result
    #         elif isinstance(raw_result, dict):
    #             result_name = raw_result.get("name")
    #             result_amount = raw_result.get("amount")
    #             raw_probability = raw_result.get("probability")
    #             if raw_probability is not None:
    #                 result_amount = result_amount * raw_probability
    #         else:
    #             print(f"Issue with results for `{name}`")
    #             continue
    #         results[result_name] = result_amount

    recipes[name] = Recipe(
        name=name,
        category=category,
        ingredients=ingredients,
        energy_required=amount_dict.get("energy_required", 0.5),
        result_count=amount_dict.get("result_count", 1),
    )

# Double check that ingredients and results are parsed correctly
for recipe_name, recipe in recipes.items():
    if recipe.ingredients == {}:
        print(f"Issue with ingredients for `{recipe_name}`")

Skipping basic-oil-processing
Skipping advanced-oil-processing
Skipping coal-liquefaction
Skipping heavy-oil-cracking
Skipping light-oil-cracking
Skipping sulfuric-acid
Skipping plastic-bar
Skipping solid-fuel-from-light-oil
Skipping solid-fuel-from-petroleum-gas
Skipping solid-fuel-from-heavy-oil
Skipping sulfur
Skipping lubricant
Skipping empty-barrel
Skipping uranium-processing
Skipping kovarex-enrichment-process
Skipping nuclear-fuel-reprocessing


In [ ]:
def build_ratio_graph(recipes: dict[str, Recipe], target: str, target_demand: float=2.5, raw_materials=set()):
    """
    Build a graph where edges show how many input machines are required per output machine.
    """
    # Get topological ordering of nodes via DFS
    visited = set()
    topo_order = []

    def dfs(product: str):
        if product in visited:
            return
        visited.add(product)
        recipe = recipes.get(product)
        is_raw_material = product in raw_materials or recipe is None
        if not is_raw_material:
            for ing in recipe.ingredients:
                dfs(ing)
        topo_order.append(product)

    dfs(target)

    # compute item demand, machine counts
    target_recipe = recipes[target]
    item_demands = defaultdict(float)
    item_demands[target] = target_demand
    machine_counts = defaultdict(float)
    machine_counts[target] = target_demand / target_recipe.result_count * target_recipe.energy_required / RECIPE_SPEEDS.get(target_recipe.category, 1.0)
    for product in reversed(topo_order):
        recipe = recipes.get(product)
        if product in raw_materials or recipe is None:
            continue  # raw material, no inputs
        machine_counts[product] = item_demands[product] / recipe.result_count * recipe.energy_required / RECIPE_SPEEDS.get(recipe.category, 1.0)
        for ing, ing_amount in recipe.ingredients.items():
            item_demands[ing] += machine_counts[product] * ing_amount / recipe.energy_required * RECIPE_SPEEDS.get(recipe.category, 1.0)

    # build edges with ratios
    edges: list[tuple[str, str, float, float]] = []
    for product in topo_order:
        recipe = recipes.get(product)
        if product in raw_materials or recipe is None:
            continue  # raw material, no inputs
        for ing, ing_amount in recipe.ingredients.items():
            machine_ratio = 0
            ing_recipe = recipes.get(ing)
            if ing_recipe:
                # TODO: Make this readable. The final ratio is the product of ratios but this expression is hard to parse.
                machine_ratio = (ing_amount / ing_recipe.result_count) * (ing_recipe.energy_required / recipe.energy_required) * (RECIPE_SPEEDS.get(recipe.category, 1.0) / RECIPE_SPEEDS.get(ing_recipe.category, 1.0))
            item_demand = machine_counts[product] * ing_amount / recipe.energy_required * RECIPE_SPEEDS.get(recipe.category, 1.0)
            edges.append((ing, product, machine_ratio, item_demand))

    # Build graphviz graph
    dot = Digraph(comment=f"{target} Dependency Ratios")

    # Nodes
    for product in topo_order:
        label_text = f"{item_demands[product]:.3g} items/s"
        if machine_counts.get(product):
            label_text += f" ({machine_counts[product]:.3g} machines)"

        icon_path = Path(ICON_DIR) / f"{product}.png"
        if icon_path.exists():
            dot.node(
                product,
                label=f"""<
                <TABLE BORDER="0" CELLBORDER="0" CELLSPACING="2">
                    <TR>
                        <TD><IMG SRC="{icon_path.resolve()}" SCALE="TRUE"/></TD>
                        <TD>{label_text}</TD>
                    </TR>
                </TABLE>
                >""",
                shape="none"
            )
        else:
            dot.node(product, label=f"{product} - {label_text}")

    # Edges
    for ing, product, machine_ratio, item_demand in edges:
        label = f"{item_demand:.3g} items/s"
        if machine_ratio:
            label += f" ({machine_ratio:.3g}:1 machines)" if machine_ratio else ""
        dot.edge(ing, product, label=label)

    return dot


name = "engine-unit"
dot = build_ratio_graph(recipes, name)

# Render graph
dot.render(f"output/{name}", format="png", cleanup=True)

name = "automation-science-pack"
dot = build_ratio_graph(recipes, name)

# Render graph
dot.render(f"output/{name}", format="png", cleanup=True)


'output/automation-science-pack.png'